In [1]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests

In [2]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from config import urls_1140412, column_names
from utils import fetch_data, combined_df, parse_admin_region, parse_sale_period, find_first_sale_time, sales_start_time

c:\PyWorkspace\presale-scraper\presale_scraper


In [3]:
import_name = 'urls_1140412'
input_time = import_name.split('_')[1]  # 提取 "1140412" 部分

df = combined_df(urls_1140412, input_time)

處理 花蓮縣 中 (找到 151 筆): 100%|██████████| 19/19 [00:43<00:00,  2.28s/it]   


各縣市資料筆數統計:
臺北市: 470 筆
新北市: 672 筆
桃園市: 1010 筆
新竹市: 206 筆
新竹縣: 410 筆
臺中市: 1242 筆
臺南市: 818 筆
高雄市: 536 筆
基隆市: 24 筆
嘉義市: 117 筆
宜蘭縣: 445 筆
苗栗縣: 420 筆
彰化縣: 538 筆
南投縣: 144 筆
雲林縣: 325 筆
嘉義縣: 178 筆
屏東縣: 189 筆
臺東縣: 97 筆
花蓮縣: 151 筆

合併後資料總筆數: 7992 筆


In [4]:
if not os.path.exists('../data'):
    os.makedirs('../data')
    
# 使用今天的日期作為檔名的一部分
today = datetime.datetime.now().strftime('%Y%m%d')
df.to_csv(f'../data/rawdata_{today}.csv', index=False, encoding='utf-8-sig')

In [5]:
df = df.rename(columns=column_names, inplace=False)

In [6]:
proc_df = df.copy()
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
proc_df["自售期間"], proc_df["代銷期間"] = zip(*proc_df["銷售期間"].apply(parse_sale_period))

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始時間"] = proc_df["自售期間"].apply(find_first_sale_time)
proc_df["代銷起始時間"] = proc_df["代銷期間"].apply(find_first_sale_time)

# 建立「銷售起始時間」欄位
proc_df["銷售起始時間"] = proc_df.apply(sales_start_time, axis=1)

In [7]:
final_df = proc_df[["縣市", "行政區", "起造人", "社區名稱", "戶數", "銷售起始時間", "銷售期間",  
"編號", "自售起始時間", "代銷起始時間", "備查完成日期", "建照核發日", "坐落街道", "主要用途", "使用分區", "建照執照", "經度", "緯度"]]
final_df

,縣市,行政區,起造人,社區名稱,戶數,銷售起始時間,銷售期間,編號,自售起始時間,代銷起始時間,備查完成日期,建照核發日,坐落街道,主要用途,使用分區,建照執照,經度,緯度
0,臺北市,大安區,潤泰創新國際股份有限公司,潤泰大安富陽,130,1100701,"自售:1100701~自完售止,1100701~售完為止;",G1A011007020007,1100701,None,1100819,1080911,大安區臥龍街268號,店舖、住宅,第三種住宅區,108建字第0168號,118.754566,0.000000
1,臺北市,大同區,中國建築經理股份有限公司,亞昕華威,130,1090601,"自售:1090601~1091231,事務所;代銷:1100501~1111130",G2A011007020003,1090601,1100501,1100825,1070103,大同區延平北路一段25號旁,住商用,商業區,107建字第0001號,121.512361,25.050456
2,臺北市,松山區,台新建築經理股份有限公司,敦仰,65,1100611,自售:無;代銷:110年10月21日起至使照取得6個月止,G2A011007050003,None,None,1120731,1100611,松山區敦化北路56號,一般零售業甲組(店舖);集合住宅,敦化南北路特定專用區(B區),110建字第0139號,121.548457,25.050931
3,臺北市,士林區,台灣新光商業銀行股份有限公司,原禾賦,27,1100701,自售:無;代銷:1100701~1101231,G1A011007050001,None,1100701,1101028,1080530,士林區德行東路335號,住商用,第三種住宅區,108建字第0100號,121.537836,25.110492
4,臺北市,大安區,富邦建設股份有限公司,富邦藝樹,60,1100301,自售:至完銷;代銷:1100301~1101031,G1A011007050002,None,1100301,1120721,1091029,大安區建安里敦化南路一段161巷5弄6號2樓,住商用,敦化南北路特定專用區(A)區,109建字第0243號,121.549528,25.044184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7987,花蓮縣,花蓮市,上寬建設不動產有限公司,上寬好室,23,1140320,自售:備查完成至售完為止;代銷:備查完成至售完為止,G2UA11403100001,None,None,1140320,1140108,花蓮市富裕三街,集合住宅,第一種商業區,花建執照字第114A0006號,121.598333,23.993766
7988,花蓮縣,新城鄉,連科金興業有限公司,日安寬庭,26,1140501,自售:無;代銷:1140501~1240501,G1UA11403200001,None,1140501,1140325,1140109,新城鄉大德街,工廠,一般農業區丁種建築用地,花建執照字第114A0015號,121.609311,24.041740
7989,花蓮縣,花蓮市,星耀建設有限公司,小倆口,16,1140325,自售:自備查完成日起至銷售完成日訖;,G1UA11403240002,None,None,1140325,1131231,花蓮市國富五街與國富八街口,住家用,住宅區,花建執照字第113A0492號,121.597978,23.989352
7990,花蓮縣,吉安鄉,德寶建設開發有限公司,北昌學府,20,1140407,自售:自備查完成後至銷售完畢;,G1UA11403250001,None,None,1140407,1130722,吉安鄉北昌三街18巷11弄3號,H2-集合住宅,住宅區,花建執照字第113A0283號,121.593886,23.986307


In [8]:
today = datetime.datetime.now().strftime('%Y%m%d')
df.to_csv(f'../data/output_{today}.csv', index=False, encoding='utf-8-sig')